In [1]:
""" Query wsdb for Gaia and GALEX magnitudes. """

##### Dependencies
import numpy as np
import pandas as pd
%matplotlib osx
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord as coord
import sqlutilpy
from astropy.io import fits

In [2]:
##### Guide to data format from Warren Brown
# ra,dec   -- J2000 coordinates (h:m:s, d:m:s)
# g0,ug0,gr0,ri0  --  de-reddened SDSS magnitude and colors (mag)
# wd,type  -- my classification (wd=1 is DA white dwarf, wd=-1 is QSO, wd=0 is B-type star)
# vel,vel_err  -- heliocentric radial velocity (km/s)
# nobs,rms   -- number of observations and RMS velocity for nobs>1 (objects with large "nobs" and "rms" are ELM WD binaries, see http://adsabs.harvard.edu/abs/2016ApJ...818..155B for a recent compilation)
# templ  -- cross-correlation template used
# v_grf  -- galacto-centric rest-frame radial velocity, calculated assuming v_circ=235 and Schonrich+10 LSR (km/s)
# glon,glat  -- Galactic longitude and latitude (deg)
# ebv  -- E(B-V) reddening value from Schlegel+98 (mag)
# Mb  -- estimate of absolute g-band mag if solar metallicty main sequence B star (mag)
# db,Rb  -- heliocentric and Galactocentric distance assuming Mb (kpc)
# Mbhb  -- estimate of absolute g-band mag if BHB star (mag)
# dbhb,Rbhb  -- heliocentric and Galactocentric distance assuming Mbhb (kpc)
# teff,teff_e,logg,logg_e  -- effective temperature and surface gravity derived from 1D pure H atmosphere models, used for white dwarfs (note the logg==5 limit:  these values are invalid in the vast majority of cases)
# u,g,r,i,z,u_e,g_e,r_e,i_e,z_e  -- SDSS apparent magnitudes and errors (mag)
# id   -- my observing identifier.  Note that a handful of objects come from SDSS spectroscopy, not mine.

##### Load in data
hvsfile = '../data/wbrown_hvs_survey.txt'
hvsdata = pd.read_fwf(hvsfile,delimiter='\s+',skiprows=[1])
hvsrastr = hvsdata['ra          '].as_matrix().astype(np.str_)
hvsdecstr = hvsdata['dec          '].as_matrix().astype(np.str_)
nhvs = len(hvsrastr)
hvswd = hvsdata['wd'].as_matrix()
hvsvel = hvsdata['vel    '].as_matrix()
hvsvelerr = hvsdata['vel_err'].as_matrix()
hvsvgrf = hvsdata['v_grf  '].as_matrix()
hvsdbhb = hvsdata['dbhb '].as_matrix()
hvsdb = hvsdata['db    '].as_matrix()
hvsug0 = hvsdata['ug0   '].as_matrix()
hvsgr0 = hvsdata['gr0    '].as_matrix()
hvsri0 = hvsdata['ri0    '].as_matrix()
hvsg0 = hvsdata['g0    '].as_matrix()
hvsnobs = hvsdata['nob'].as_matrix()
hvsebv = hvsdata['ebv   '].as_matrix()
hvstype = hvsdata['type'].as_matrix()

##### Sanitize data
for i in range(nhvs):
    hvsrastr[i] = hvsrastr[i].strip(' ')
    hvsdecstr[i] = hvsdecstr[i].strip(' ')
    if (hvsdecstr[i][0] != '-') & (hvsdecstr[i][0] != '+'):
        hvsdecstr[i] = '+' + hvsdecstr[i]
hvsvelerr = np.array([float(val) if val != '       ' else np.nan for val in hvsvelerr])
hvsvel = np.array([float(val) if val != '       ' else np.nan for val in hvsvel])
hvsdbhb = np.array([float(val) if val != '     ' else np.nan for val in hvsdbhb])
hvsvgrf = np.array([float(val) if val != '       ' else np.nan for val in hvsvgrf])

##### Obtain numeric ra/dec
hvsradec = np.array(np.char.array(hvsrastr)+np.char.array(hvsdecstr))
hvscoord = coord(hvsradec,unit=(u.hourangle, u.deg))
hvsra = hvscoord.ra.deg
hvsdec = hvscoord.dec.deg
hvsl = hvscoord.galactic.l.deg
hvsb = hvscoord.galactic.b.deg

In [4]:
##### Query WSDB to get Gaia data 
hvs_keys = ['hvsra', 'hvsdec', 'hvsl', 'hvsb', 'hvsvel', 'hvsvelerr', 'hvswd', 'hvstype', 'hvsnobs', 'hvsebv', 'hvsdb', 'hvsdbhb', 'hvsg0', 'hvsug0', 'hvsgr0', 'hvsri0']
hvs_x = [hvsra, hvsdec, hvsl, hvsb, hvsvel, hvsvelerr, hvswd, hvstype, hvsnobs, hvsebv, hvsdb, hvsdbhb, hvsg0, hvsug0, hvsgr0, hvsri0]
gaia_keys = ['source_id', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_flux_over_error', 'phot_g_mean_mag', 'phot_bp_n_obs', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag', 'phot_rp_n_obs', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag', 'phot_bp_rp_excess_factor', 'phot_proc_mode', 'bp_rp', 'bp_g', 'g_rp', 'phot_variable_flag', 'l', 'b', 'a_g_val', 'e_bp_min_rp_val', 'radial_velocity', 'radial_velocity_error', 'rv_nb_transits','rv_template_teff','rv_template_logg','rv_template_fe_h','ebv']
galex_keys = ['galex_ra','galex_dec','fuv_mag','fuv_magerr','nuv_mag','nuv_magerr']
allwise_keys = ['allwise_ra','allwise_dec','w1mpro','w1sigmpro','w2mpro','w2sigmpro']

In [5]:
### Cross-match with Gaia source
gaia_x = sqlutilpy.local_join("""
   select CAST(source_id as real), ra, ra_error, dec, dec_error, parallax, parallax_error, parallax_over_error, pmra, pmra_error, pmdec, pmdec_error, ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr, parallax_pmra_corr, parallax_pmdec_corr, pmra_pmdec_corr, CAST(astrometric_n_obs_al as real), CAST(astrometric_n_obs_ac as real), CAST(astrometric_n_good_obs_al as real), CAST(astrometric_n_bad_obs_al as real), astrometric_gof_al, astrometric_chi2_al, astrometric_excess_noise, astrometric_excess_noise_sig, CAST(astrometric_params_solved as real), astrometric_primary_flag, astrometric_weight_al, astrometric_pseudo_colour, astrometric_pseudo_colour_error, mean_varpi_factor_al, CAST(astrometric_matched_observations as real), CAST(visibility_periods_used as real), astrometric_sigma5d_max, CAST(frame_rotator_object_type as real), CAST(matched_observations as real), duplicated_source, CAST(phot_g_n_obs as real), phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_flux_over_error, phot_g_mean_mag, CAST(phot_bp_n_obs as real), phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, CAST(phot_rp_n_obs as real), phot_rp_mean_flux, phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, phot_bp_rp_excess_factor, CAST(phot_proc_mode as real), bp_rp, bp_g, g_rp, phot_variable_flag, l, b, a_g_val, e_bp_min_rp_val, radial_velocity, radial_velocity_error, CAST(rv_nb_transits as real),rv_template_teff,rv_template_logg,rv_template_fe_h,ebv from mytable as m left join lateral (select * from gaia_dr2.gaia_source as g where 
               q3c_join(m.hvsra, m.hvsdec,g.ra,g.dec,1./3600)
       order by q3c_dist(m.hvsra,m.hvsdec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (hvsra,hvsdec,np.arange(len(hvsra))),('hvsra','hvsdec','xid'),
           host='cappc127.ast.cam.ac.uk', user='douglas_boubert', password='chopel3082',
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished Gaia cross-match, "+str(np.where(np.isnan(gaia_x[0])==False)[0].shape[0])+" sources.")                

### Cross-match with GALEX
galex_x = sqlutilpy.local_join("""
   select ra,dec,fuv_mag,fuv_magerr,nuv_mag,nuv_magerr from mytable as m left join lateral (select * from galex_bst17.main_fov055 as g where 
               q3c_join(m.hvsra, m.hvsdec,g.ra,g.dec,10./3600)
       order by q3c_dist(m.hvsra,m.hvsdec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (hvsra,hvsdec,np.arange(len(hvsra))),('hvsra','hvsdec','xid'),
           host='cappc127.ast.cam.ac.uk', user='douglas_boubert', password='chopel3082',
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished GALEX cross-match, "+str(np.where(np.isnan(galex_x[0])==False)[0].shape[0])+" sources.")

### Cross-match with ALLWISE
allwise_x = sqlutilpy.local_join("""
   select ra, dec, w1mpro, w1sigmpro, w2mpro, w2sigmpro from mytable as m left join lateral (select * from allwise.main as g where 
               q3c_join(m.hvsra, m.hvsdec,g.ra,g.dec,15./3600)
       order by q3c_dist(m.hvsra,m.hvsdec,g.ra,g.dec) asc limit 1)
       as tt on  true  order by xid""",'mytable',
       (hvsra,hvsdec,np.arange(len(hvsra))),('hvsra','hvsdec','xid'),
           host='cappc127.ast.cam.ac.uk', user='douglas_boubert', password='chopel3082',
       preamb='set enable_seqscan to off; set enable_mergejoin to off; set enable_hashjoin to off;')
print("Finished ALLWISE cross-match, "+str(np.where(np.isnan(allwise_x[0])==False)[0].shape[0])+" sources.") 

Finished Gaia cross-match, 1427 sources.
Finished GALEX cross-match, 1104 sources.
Finished ALLWISE cross-match, 999 sources.


In [6]:
##### Merge together
x = hvs_x+gaia_x+galex_x+allwise_x
keys = hvs_keys+gaia_keys+galex_keys+allwise_keys
gagahvsbox = {keys[i]:x[i] for i in range(len(x))}

In [7]:
##### Save cross-match
np.savez_compressed('/Users/douglasboubert/Documents/Science/GaiaDR2/HVSsurvey/data/gagahvs.npz',gagahvsbox=gagahvsbox)